In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_columns', None)

# Red Wine Casus


We gaan aan de slag met een dataset over Portugese rode wijnen. Aan de hand van de gegeven dataset gaan wij een aantal onderzoeksvragen opstellen en beantwoorden.

**Onderzoeksvragen**
1. In hoeverre is de score van een Portugese Red te voorspellen op basis van de chemische kenmerken?(verplichte onderzoeksvraag) (Supervised - Regressie)
2. Als we wijnen categoriseren op basis van de chemische samenstelling, zijn er bepaalde categorieën die mannelijke proevers anders beoordelen dan vrouwelijke proevers? (Unsupervised - Clustering)
3. Kun je op basis van bepaalde keywords in de beschrijving een voorspelling doen over hoe hoog de score van deze wijn is? (Supervised - Regressie)

In de tabel die hieronder weergegeven wordt zien we de eerste 5 records die deze dataset bevat. Onder de tabel hebben we per kolom een beschrijving gegeven van wat deze kolom precies inhoudt.

In [ ]:
wine = pd.read_csv('redwine.csv', delimiter=';')
wine.head()

__Beschrijving van de kolommen__
<li>Country: Het land van herkomst van de wijn. In deze dataset komt elke wijn uit Portugal, het opslaan van deze kolom is dus redelijk onnodig aangezien het bij elk record hetzelfde is.</li>
<li>Description: Een tekstuele beschrijving over de wijn.</li>
<li>Designation: Vanuit waar de wijn verkocht wordt.</li>
<li>Points: De score van de wijn op een schaal van 1-100.</li>
<li>Price: De prijs van de wijn.</li>
<li>Province: De provincie waar de wijn is gemaakt.</li>
<li>Taster_name: De naam van de persoon die de wijn geproeft heeft.</li>
<li>Title: De naam van de wijn</li>
<li>Variety: Wijnsoort. Ook hier valt het op dat alle wijnen hierbij dezelfde waarde hebben</li>
<li>Winery: De wijnmakerij waar de wijn vandaan komt.</li>

_Chemische samenstelling_
<li>Fixed acidity: vaste zuurtegraad</li>
<li>Volatile acidity: vluchtige zuurtegtraad</li>
<li>Citric acid: citroenzuur</li>
<li>Residual sugar: Natuurlijke druifsuikers die achterblijven in de wijn nadat de alcoholische fermentatie plaats heeft gevonden.</li>
<li>Chlorides: zout</li>
<li>Free sulfur dioxide: ongebonden sulfur dioxide</li>
<li>Total sulfur dioxide: totaal aantal sulfur dioxide in de wijn</li>
<li>Density: dichtheid</li>
<li>pH: pH-waarde</li>
<li>Sulphates: sulfaat</li>
<li>alcohol: alcohol percentage</li>

# Dataverkenning: Wat viel ons op?

We begonnen met het bekijken van de dataset en het documenteren van opvallende waardes. Deze data verkennings fase is belangrijk om later te kunnen beslissen welke gegevens wel of niet bruikbaar zijn om onze onderzoeksvragen te kunnen beantwoorden. 

Zo zie je bijvoorbeeld dat het gemiddelde van het aantal punten van de wijnen 88.8 is. Dit zou dus op een schaal van 1-10 uitkomen op een 8,8. 

In [ ]:
np.mean(wine['points'])

Ook zie je dat het laagst gegeven cijfer een 8,1 is. Redelijk enthousiast. 

In [ ]:
np.min(wine['points'])

Er zijn ook een aantal records waarbij lege waardes voorkomen. We moeten hierbij kijken wat we gaan doen met de lege data en of het invloed heeft op de antwoorden van onze onderzoeksvragen. 

In [ ]:
wine.isnull().sum().sum()

De tabel hieronder toont aan dat alle tasters meer dan één wijnsoort geproeft hebben. Je ziet zelfs dat sommigen er honderden hebben beoordeeld. 

In [ ]:
nameCount = wine.groupby(['taster_name']).size().reset_index(name='count')
nameCount

# Externe dataset

We willen een externe dataset gebruiken voor het beantwoorden van een van onze onderzoeksvragen. De dataset bestaat uit engelse namen en hun geslacht. Aan de hand van de informatie uit deze dataset willen we voorspellen of de tasters in onze eigen dataset van het mannelijk of vrouwelijke geslacht zijn. 


Hieronder zien we de eerste 10 records van de externe dataset. 

In [ ]:
name = pd.read_csv('names.csv', usecols=['Name','Gender'])
name.head(10)

In [ ]:
wine['taster_name'] = wine['taster_name'].apply(lambda name: name.split(" ")[0])

## De chemische samenstelling.

In [ ]:
chemColNames = ['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']
chem = wine[chemColNames]
chem.head()
# citric acid, density, alcohol

In [ ]:
chem.describe(include='all')

Hier zie je bijvoorbeeld het vershil in spreiding, wat opvalt is dat er in suiker maar vooral in sulfur een groot aantal hoge uitschieters zijn.

In [ ]:
chem.boxplot(figsize=(20,10))

# Pair plot

Voordat we een pairplot kunnen maken moeten we de types van een aantal kolommen aanpassen. We willen bijvoorbeeld alcohol plotten en daarvoor moeten we eerst zorgen dat alcohol van het type object naar een nummeriek type geconverteerd wordt. We kunnen terwijl we dit doen ook meteen de waardes van density en citric acid omzetten naar nummerieke datatypes, mogelijk hebben we deze later nog nodig. 

In [ ]:
colErrorPairs = {
    'density'    : [' . '],
    'citric acid': [' - ',' -   '],
    'alcohol'    : ['100.333.333.333.333','11.066.666.666.666.600','956.666.666.666.667','923.333.333.333.333']}

for colName in colErrorPairs:
    for faultyString in colErrorPairs[colName]:
        wine[colName] = wine[colName].replace(faultyString,np.nan)

In [ ]:
wine['alcohol'] = wine['alcohol'].astype(float)

In [ ]:
wine['density'] = wine['density'].astype(float)

In [ ]:
wine['citric acid'] = wine['citric acid'].astype(float)

In [ ]:
wine.dtypes

#### Nu dat we de gewenste types hebben omgezet kunnen we ze plotten in een pairplot

In [ ]:
sns.set(style="ticks", color_codes=True)
g = sns.pairplot(wine, diag_kind = "hist",
                 height = 4,
                 hue = "taster_name",

                 x_vars=["points", "price", "alcohol"], 
                 y_vars= ["points", "price", "alcohol"])

plt.show()

Hierboven zien we een pairplot waarbij we de variabelen prijs, points en alcohol met elkaar vergelijken. Elke proever heeft een apart kleurtje gekregen. 

# Standaardiseren


Stanaardiseren helpt om eerlijker te meten. Je verwijdert de eenheden en zorgt dat alle variabelen ongeveer hetzeflde bereik hebben. 

Alle variabelen die te maken hebben met de chemische samenstelling zouden ongeveer hetzelfde bereik moeten hebben. Als we ervoor zorgen dat deze waardes bijvoorbeeld allemaal tussen de 0 en 1 liggen, kunnen we ze makkelijk met elkaar vergelijken zonder dat er één veel zwaarder weegt dan de ander.


In [ ]:
wineZscore = wine
for col in chemColNames:
        wineZscore[col] = (wine[col] - wine[col].mean())/wine[col].std(ddof=0)
wineZscore.head()

In [ ]:
wineZscore.boxplot(figsize=(20,10), column=chemColNames)

# Onderzoeksvraag 1


Voor de eerste verplichte onderzoeksvraag gaan we supervised machine learning gebruiken. We willen de score van een wijn gaan voorspellen aan de hand van de chemische samenstelling. Dit gaan we doen met behulp van lineaire regressie. 

__1. Kies een modeltype__ 

__2. Kies de hyperparameters__


__3. Organiseer de data (feature matrix, target vector)__

In [ ]:
features = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates']

Voor het toepassen van lineaire regressie beginnen we met het definieren van de feature variabelen. Dit zijn in dit geval alle variabelen die te maken hebben met de chemische samenstelling. Op basis van deze features willen we namelijk een voorspelling gaan doen. In de onderstaande tabel zien we de features met hun eerste 5 records. 

In [ ]:
X = wine[features]
X.head()

In [ ]:
X.plot.hist()

In [ ]:
print(type(X))
print(X.shape)

Nu gaan we de target variabele specificeren. In dit geval is dit points. We willen namelijk het aantal points gaan voorspellen. 

In [ ]:
y = wine['points']
y.head()

In [ ]:
print(type(y))
print(y.shape)

__4. Creeër een training- en validatie set.__

In [ ]:
X.fillna(X.mean(),inplace=True)
y.fillna(y.mean(),inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

We maken een training en test set aan. Hierna 

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

__5. Start het leerproces met fit()__

In [ ]:
linreg = LinearRegression()

In [ ]:
linreg.fit(X_train, y_train)

In [ ]:
linreg.score(X_test,y_test)

In [ ]:
y_pred = linreg.predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_test,y_pred))